In [14]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# BLEU
import evaluate

# Cosine Similarity
from sentence_transformers import SentenceTransformer

In [9]:
paraphrased_eval_df = pd.read_csv("data/paraphrased_eval.csv")
paraphrased_eval_df.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate',
       'question1_token_len', 'question2_token_len', 'synonym_replacement',
       'finetuned_t5', 'original_t5', 'gpt_t5', 'llm'],
      dtype='object')

# BLEU score

In [ ]:
# Check here: https://huggingface.co/spaces/evaluate-metric/bleu
references =  paraphrased_eval_df["question2"].to_list()
references = [[reference] for reference in references]

synonym_replacement = paraphrased_eval_df["synonym_replacement"].to_list()
finetuned_t5 = paraphrased_eval_df["finetuned_t5"].to_list()
gpt_t5 = paraphrased_eval_df["gpt_t5"].to_list()
original_t5 = paraphrased_eval_df["original_t5"].to_list()
llm = paraphrased_eval_df["llm"].to_list()


bleu = evaluate.load("bleu")

synonym_replacement_bleu = bleu.compute(references=references, predictions=synonym_replacement)
finetuned_t5_bleu = bleu.compute(references=references, predictions=finetuned_t5)
gpt_t5_bleu = bleu.compute(references=references, predictions=gpt_t5)
original_t5_bleu = bleu.compute(references=references, predictions=original_t5)
llm_bleu = bleu.compute(references=references, predictions=llm)

print("Synonym Replacement BLEU: ", synonym_replacement_bleu)
print("Finetuned T5 BLEU: ", finetuned_t5_bleu)
print("GPT T5 BLEU: ", gpt_t5_bleu)
print("Original T5 BLEU: ", original_t5_bleu)
print("LLM BLEU: ", llm_bleu)

Synonym Replacement BLEU:  {'bleu': 0.18972922957262872, 'precisions': [0.5110855168442269, 0.2477150961235424, 0.13505046989209885, 0.0757870190439176], 'brevity_penalty': 1.0, 'length_ratio': 1.0178780773739742, 'translation_length': 3473, 'reference_length': 3412}
Finetuned T5 BLEU:  {'bleu': 0.1947631910277385, 'precisions': [0.541747572815534, 0.2756272401433692, 0.15542168674698795, 0.09406392694063927], 'brevity_penalty': 0.9010386552002413, 'length_ratio': 0.9056271981242673, 'translation_length': 3090, 'reference_length': 3412}
GPT T5 BLEU:  {'bleu': 0.10706842303409357, 'precisions': [0.39901732609257823, 0.15503223997757218, 0.07162534435261708, 0.02965958881024604], 'brevity_penalty': 1.0, 'length_ratio': 1.1333528722157094, 'translation_length': 3867, 'reference_length': 3412}
Original T5 BLEU:  {'bleu': 0.22917713814817564, 'precisions': [0.5704948646125116, 0.30518365945760384, 0.18484500574052812, 0.10981409424989191], 'brevity_penalty': 0.9399431488768678, 'length_rati

- Taken from: https://huggingface.co/spaces/evaluate-metric/bleu
- BLEU compares overlap in tokens from the predictions and references, instead of comparing meaning. This can lead to discrepancies between BLEU scores and human ratings.
- Shorter predicted translations achieve higher scores than longer ones, simply due to how the score is calculated. A brevity penalty is introduced to attempt to counteract this.
- BLEU scores are not comparable across different datasets, nor are they comparable across different languages.
- BLEU scores can vary greatly depending on which parameters are used to generate the scores, especially when different tokenization and normalization techniques are used. It is therefore not possible to compare BLEU scores generated using different parameters, or when these parameters are unknown. For more discussion around this topic, see the following issue.

# Cosine Similarity

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

def cosine_similarity_pairwise(a, b):
    # Normalize the vectors
    a_norm = a / np.linalg.norm(a, axis=1, keepdims=True)
    b_norm = b / np.linalg.norm(b, axis=1, keepdims=True)
    # Calculate dot product for corresponding pairs
    return np.sum(a_norm * b_norm, axis=1)

references = paraphrased_eval_df["question2"].to_list()
synonym_replacement = paraphrased_eval_df["synonym_replacement"].to_list()
finetuned_t5 = paraphrased_eval_df["finetuned_t5"].to_list()
gpt_t5 = paraphrased_eval_df["gpt_t5"].to_list()
original_t5 = paraphrased_eval_df["original_t5"].to_list()
llm = paraphrased_eval_df["llm"].to_list()

references_emb = model.encode(references)
synonym_replacement_emb = model.encode(synonym_replacement)
finetuned_t5_emb = model.encode(finetuned_t5)
gpt_t5_emb = model.encode(gpt_t5)
original_t5_emb = model.encode(original_t5)
llm_emb = model.encode(llm)


# Calculate pairwise similarities
synonym_replacement_similarities = cosine_similarity_pairwise(synonym_replacement_emb, references_emb)
finetuned_t5_similarities = cosine_similarity_pairwise(finetuned_t5_emb, references_emb)
gpt_t5_similarities = cosine_similarity_pairwise(gpt_t5_emb, references_emb)
original_t5_similarities = cosine_similarity_pairwise(original_t5_emb, references_emb)
llm_similarities = cosine_similarity_pairwise(llm_emb, references_emb)

synonym_replacement_cosine_similarity = synonym_replacement_similarities.mean()
finetuned_t5_cosine_similarity = finetuned_t5_similarities.mean()
gpt_t5_cosine_similarity = gpt_t5_similarities.mean()
original_t5_cosine_similarity = original_t5_similarities.mean()
llm_cosine_similarity = llm_similarities.mean()

print("Synonym Replacement Cosine Similarity: ", synonym_replacement_cosine_similarity)
print("Finetuned T5 Cosine Similarity: ", finetuned_t5_cosine_similarity)
print("GPT T5 Cosine Similarity: ", gpt_t5_cosine_similarity)
print("Original T5 Cosine Similarity: ", original_t5_cosine_similarity)
print("LLM Cosine Similarity: ", llm_cosine_similarity)

Synonym Replacement Cosine Similarity:  0.68806964
Finetuned T5 Cosine Similarity:  0.8130487
GPT T5 Cosine Similarity:  0.7879094
Original T5 Cosine Similarity:  0.8381208
LLM Cosine Similarity:  0.7537029
